# ETL Pipeline

## 1. Import Libraries 

In [77]:
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

## 2. Define Data Processing Class

In [52]:
class DataProcessing:
    def __init__(self,df):
        '''
        Input
        df: datafame
        Initialize the class instance object variable df with data frame
        '''
        self.df = df
    
    def convert_to_category(self,column):
        '''
        Input: column whose values needs to be converted to binary category columns
        Example: This column value "related-1;request-0;offer-0;aid_related-0"
        will be converted to 4 different columns having column names 
        ('related','request','offer','aid_related') with values (1,0,0,0)
        '''
        categories = (self.df[column]).str.split (pat = ';', expand = True)
        cat_cols = categories.iloc[0].apply(lambda x: x.rstrip('- 0 1'))
        categories.columns = cat_cols

        for col in cat_cols:
            categories[col] = categories[col].apply(lambda x: int(x[-1]))
        self.df.drop ([column], axis = 1, inplace = True)
        self.df = pd.concat ([self.df, categories], axis = 1, sort = False)
    
    def remove_duplicates(self,col):
        '''
        Input 
        col: column name
        All the rows having duplicate values for that column are dropped
        '''
        self.df.drop_duplicates(subset=[col],inplace=True)
    
    def merge_df(self,df2,col):
        '''
        Input
        df2: Second data frame with which self.df needs to be merged
        col: column name on which the merge has to happen
        '''
        self.df = self.df.merge (df2, left_on = col, right_on = col, how = 'inner', validate = 'many_to_many')

        
    def drop_column(self,col):
        '''
        Input
        col: column name that has to be dropped
        '''
        self.df.drop(columns=[col],inplace=True)
    
    def drop_row_column_value(self,col,val):
        '''
        Input
        col: column name
        val: value that needs to be checked and the corresponding rows has to be removed
        '''
        self.df.drop(self.df[self.df['related']==2].index,inplace=True)
        

## 3. Read CSV

In [53]:
df_cat = pd.read_csv('disaster_categories.csv')
df_mes = pd.read_csv('disaster_messages.csv')

### Let's look at both the dataset

In [54]:
df_cat.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [55]:
df_cat.shape

(26248, 2)

In [56]:
df_mes.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [57]:
df_mes.shape

(26248, 4)

## Let's print number of unique values

In [58]:
print(df_cat['id'].nunique(),df_mes['id'].nunique())

26180 26180


## Check if there is any null values in any columns

In [59]:
df_cat.isnull().sum()

id            0
categories    0
dtype: int64

In [60]:
df_mes.isnull().sum()

id              0
message         0
original    16064
genre           0
dtype: int64

## Let's check duplicate rows in the data set

In [61]:
df_mes[df_mes.duplicated()].sort_values(by=['id'])

,id,message,original,genre
163,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct
656,804,elle est vraiment malade et a besoin d'aide. u...,she is really sick she need your help. please ...,direct
710,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct
1408,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct
2131,2446,How much money did TV Latino American collect ...,Konbyen kob tv latino america ranmase miami,direct
...,...,...,...,...
24040,27768,An EU-backed French offensive against rebels r...,NaN,news
24648,28462,"To date, a total of 39 people suffering from E...",NaN,news
24856,28687,"At present, UNHCR has 2,500 additional tents a...",NaN,news
25156,29022,"In a field in Jallouzai, just inside Pakistan,...",NaN,news


In [62]:
ids = df_cat['categories']
display (df_cat [ids.isin (ids [ids.duplicated ()])].sort_values (by = ['id']).head (n=5))

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...
5,14,related-0;request-0;offer-0;aid_related-0;medi...


## Now we do some cleaning in the dataset

In [63]:
# Initializing the class object for data processing
data_process = DataProcessing(df_cat)

In [64]:
# First we will clean the df_cat data
# Here we will convert column values into binary label columns
data_process.convert_to_category('categories')

In [65]:
# Here we are removing the column named 'child_alone' as it has all values are same.
data_process.drop_column('child_alone')

In [66]:
data_process.df.head(10)

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,1,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,1,1,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [67]:
# Merger the 2 dataframes on id.
data_process.merge_df(df_mes,'id')

In [68]:
data_process.df.columns

Index(['id', 'related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report',
       'message', 'original', 'genre'],
      dtype='object')

In [69]:
# Remove duplicate rows havin same message
data_process.remove_duplicates('message')

In [70]:
data_process.df.shape

(26177, 39)

In [71]:
data_process.df.nunique()

id                        26177
related                       3
request                       2
offer                         2
aid_related                   2
medical_help                  2
medical_products              2
search_and_rescue             2
security                      2
military                      2
water                         2
food                          2
shelter                       2
clothing                      2
money                         2
missing_people                2
refugees                      2
death                         2
other_aid                     2
infrastructure_related        2
transport                     2
buildings                     2
electricity                   2
tools                         2
hospitals                     2
shops                         2
aid_centers                   2
other_infrastructure          2
weather_related               2
floods                        2
storm                         2
fire    

From the above you can see that related has 3 unique values Let's check frequency for each value

In [72]:
data_process.df['related'].value_counts()

1    19874
0     6115
2      188
Name: related, dtype: int64

In [73]:
data_process.drop_row_column_value('related',2)

In [74]:
data_process.df

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,floods,storm,fire,earthquake,cold,other_weather,direct_report,message,original,genre
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26381,30261,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,The training demonstrated how to enhance micro...,NaN,news
26382,30262,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,A suitable candidate has been selected and OCH...,NaN,news
26383,30263,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"Proshika, operating in Cox's Bazar municipalit...",NaN,news
26384,30264,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,"Some 2,000 women protesting against the conduc...",NaN,news


In [75]:
## Save data into sqlite database

In [79]:
engine = create_engine('sqlite:///DisasterResponse1.db')
data_process.df.to_sql('message_and_category', engine, index=False, if_exists = 'replace')